### Получение ID частного телеграмм-канала
- Создать открытый телеграмм-канал
- Присвоить переменной **url** адрес канала
- **Добавить в администраторы бота!**
- Выполнить запрос
- Сделать канал частным
- Скопировать **id** в конфигурационный файл


In [ ]:
import bot_settings as bs
import requests
import json


url = 'https://t.me/gftfds'
channel_name = '@{}'.format(url.split('/')[-1])

response = requests.get('https://api.telegram.org/bot{}/getChat'.format(bs.token), params=dict(
    chat_id=channel_name))

data = response.json()
print('id = ', data['result']['id'])